<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_3_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
Unpacking objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-3.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-3.validation.csv",header=None)
valid.head()

,0,1
0,91,jitters
1,93,shooting
2,187,dry mouth
3,189,dizzy
4,230,mania


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-3.test.csv",header=None)
test.head()

,0,1
0,13,insomniac
1,16,couldn't taste anything
2,61,spins
3,85,tremor
4,94,anxiety


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.214527,4.628829,0.237063,15:29


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.520103,4.337759,0.261859,15:25


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.223627,4.302651,0.264790,15:22


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.187290,4.277904,0.267178,15:20


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.121956,6.501894,0.126050,00:22


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.230697,5.200871,0.193277,00:26


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.688677,4.947961,0.243697,01:06


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.480513,4.464542,0.243697,01:08
1,5.246855,4.372626,0.252101,01:07


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.989658,4.144042,0.243697,01:09
1,4.838740,4.060498,0.277311,01:05


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.639925,3.870271,0.294118,01:09
1,4.479494,3.836408,0.277311,01:08


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.311956,3.816767,0.310924,01:04
1,4.139282,3.781370,0.294118,01:07


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.009859,3.617385,0.352941,01:07
1,3.872586,3.609086,0.361345,01:09


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.738623,3.558254,0.336134,01:07
1,3.676865,3.571925,0.352941,01:07
2,3.497378,3.502365,0.344538,01:09
3,3.338898,3.467515,0.352941,01:09


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.166057,3.373985,0.403361,01:09
1,3.157438,3.551049,0.369748,01:06
2,2.989246,3.494457,0.369748,01:10
3,2.861301,3.494229,0.378151,01:04


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.749481,3.483648,0.403361,01:04
1,2.744806,3.435764,0.420168,01:08
2,2.581568,3.473252,0.378151,01:05
3,2.465383,3.403143,0.394958,01:09


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.299229,3.451520,0.411765,01:09
1,2.349592,3.430415,0.394958,01:05
2,2.232034,3.507697,0.394958,01:10
3,2.114117,3.492615,0.394958,01:07


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.030721,3.462795,0.378151,01:07
1,2.029195,3.713450,0.394958,01:06
2,1.946986,3.618079,0.378151,01:04
3,1.836493,3.579865,0.378151,01:04


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.729539,3.701578,0.361345,01:06
1,1.766562,3.606872,0.403361,01:11
2,1.729772,3.612729,0.394958,01:06
3,1.666990,3.621644,0.386555,01:07


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.548688,3.880399,0.369748,01:06
1,1.620286,3.805822,0.361345,01:07
2,1.643890,4.005485,0.352941,01:04
3,1.539789,3.857682,0.378151,01:09


In [45]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.544880,3.934446,0.394958,01:08
1,1.525304,3.949235,0.361345,01:07


In [50]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.516727,3.963177,0.378151,01:11
1,1.435311,4.011669,0.344538,01:09


In [55]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.472629,4.098217,0.361345,01:08


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [58]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1701, 1962, 1599, 1204, 978, 750, 1062, 1054, 189, 1007, 233, 249, 1619, 1599, 1727, 392, 1054, 231, 13, 393, 393, 1054, 2026, 2118, 2063, 232, 1199, 1230, 1599, 1483, 1204, 749, 749, 749, 974, 792, 792, 974, 809, 809, 809, 1308, 418, 1599, 1973, 605, 1727, 1599, 974, 974, 1676, 232, 1484, 1669, 1054, 825, 1599, 1099, 1925, 1136, 1136, 1145, 749, 1204, 1054, 1340, 1265, 1599, 189, 189, 2149, 1597, 1597, 1942, 1508, 749, 1599, 1484, 1787, 2147, 452, 1599, 1099, 1599, 418, 1783, 1610, 1597, 1599, 1610, 1099, 974, 1599, 1599, 1507, 1711, 1711, 1099, 2097, 1099, 2191, 1767, 1767, 1767, 1787, 13, 1787, 393, 1787, 2150, 393, 393, 1787, 1815, 1822, 1599, 825, 825, 792, 1915, 1925, 1936, 1936, 749, 1942, 858, 2032, 393, 1787, 13, 1599, 2081, 1599, 224, 2147, 2147, 2147, 233, 2147, 1054, 1599, 907, 1718]
[13, 16, 61, 85, 94, 97, 109, 121, 189, 203, 233, 249, 265, 276, 284, 309, 359, 392, 393, 393, 393, 426, 441, 452, 452, 452, 561, 579, 629, 712, 739, 749, 749, 749, 792, 792, 792, 792,

In [59]:
correct_num = 0

for i in range(0, 143):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

47
0.32867132867132864
